In [1]:
import os
import re
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime

# Working sample data

In [2]:
sample = """
ID  FA

1  Eric Wright and Frank Robertson, 2008         
2  John Long, Lynn Hill & Joanne Urioste '83    
3  L. Hill, 1993
"""

In [3]:
sample.strip().splitlines()

['ID  FA',
 '',
 '1  Eric Wright and Frank Robertson, 2008         ',
 "2  John Long, Lynn Hill & Joanne Urioste '83    ",
 '3  L. Hill, 1993']

In [4]:
header = sample.strip().splitlines()[0].split("  ")
header

['ID', 'FA']

In [5]:
rows = [r.split("  ")[:2] for r in sample.strip().splitlines()[1:] if r]
rows

[['1', 'Eric Wright and Frank Robertson, 2008'],
 ['2', "John Long, Lynn Hill & Joanne Urioste '83"],
 ['3', 'L. Hill, 1993']]

# Built init dataframe

In [6]:
df = pd.DataFrame(rows, columns=header)

In [7]:
df.set_index('ID', inplace=True)

In [8]:
df.columns = ['fa']

In [9]:
df

,fa
ID,
1,"Eric Wright and Frank Robertson, 2008"
2,"John Long, Lynn Hill & Joanne Urioste '83"
3,"L. Hill, 1993"


In [10]:
df

,fa
ID,
1,"Eric Wright and Frank Robertson, 2008"
2,"John Long, Lynn Hill & Joanne Urioste '83"
3,"L. Hill, 1993"


# Feature Extraction

In [11]:
df['fa']

ID
1        Eric Wright and Frank Robertson, 2008
2    John Long, Lynn Hill & Joanne Urioste '83
3                                L. Hill, 1993
Name: fa, dtype: object

## Extract Year

In [12]:
df['year'] = df['fa'].str.extract(r"(\d+)\s*$", expand=False)
df

,fa,year
ID,,
1,"Eric Wright and Frank Robertson, 2008",2008
2,"John Long, Lynn Hill & Joanne Urioste '83",83
3,"L. Hill, 1993",1993


In [13]:
df['fa_text'] = df.apply(lambda x: 
                     x['fa'][::-1].replace(x['year'][::-1], "", 1)[::-1], 
                 axis=1) \
                .str.strip(", '")

df

,fa,year,fa_text
ID,,,
1,"Eric Wright and Frank Robertson, 2008",2008,Eric Wright and Frank Robertson
2,"John Long, Lynn Hill & Joanne Urioste '83",83,"John Long, Lynn Hill & Joanne Urioste"
3,"L. Hill, 1993",1993,L. Hill


## Split Climbers

In [14]:
# split climbers by all possible separators
# expand them in rows 
# clean them
# join them all back by ";"
df['climbers'] = df['fa_text'] \
    .str.split(" and |&|,") \
    .explode() \
    .str.strip() \
    .groupby(['ID']).apply(lambda x: ';'.join(x.astype(str)))
df['climbers']

ID
1           Eric Wright;Frank Robertson
2    John Long;Lynn Hill;Joanne Urioste
3                               L. Hill
Name: climbers, dtype: object

In [15]:
df

,fa,year,fa_text,climbers
ID,,,,
1,"Eric Wright and Frank Robertson, 2008",2008,Eric Wright and Frank Robertson,Eric Wright;Frank Robertson
2,"John Long, Lynn Hill & Joanne Urioste '83",83,"John Long, Lynn Hill & Joanne Urioste",John Long;Lynn Hill;Joanne Urioste
3,"L. Hill, 1993",1993,L. Hill,L. Hill


## Normalize climber names

Prepare a manual decision dict to rename climbers names as much as possible

In [16]:
df['climbers'].str.split(";", expand=False)\
    .explode() \
    .sort_values()

ID
1        Eric Wright
1    Frank Robertson
2     Joanne Urioste
2          John Long
3            L. Hill
2          Lynn Hill
Name: climbers, dtype: object

In [17]:
renamings = {k: k for k in 
                df['climbers'].str.split(";", expand=False)\
                .explode() \
                .sort_values().unique()}
renamings

{'Eric Wright': 'Eric Wright',
 'Frank Robertson': 'Frank Robertson',
 'Joanne Urioste': 'Joanne Urioste',
 'John Long': 'John Long',
 'L. Hill': 'L. Hill',
 'Lynn Hill': 'Lynn Hill'}

Copy and paste above and manual edit as desired.
This is probably most suitable step to take in Prophecies application if it becomes pretty big task.

In [18]:
my_manual_renamings = {
    #'Eric Wright': 'Eric Wright',
    #'Frank Robertson': 'Frank Robertson',
    #'Joanne Urioste': 'Joanne Urioste',
    #'John Long': 'John Long',
    'L. Hill': 'Lynn Hill',
    #'Lynn Hill': 'Lynn Hill'
}

In [19]:
def run_all_renamings(x):
    for k, v in my_manual_renamings.items():
        x = x.replace(k, v)
    return x

In [20]:
df['climbers'] = df['climbers'].map(run_all_renamings)

## Expand Climbers in columns

In [21]:
dft = df['climbers'].str.split(";", expand=True)
dft.columns = ['climber_' + str(i+1) for i in dft.columns]
df = pd.concat([df, dft], axis=1)

In [22]:
df

,fa,year,fa_text,climbers,climber_1,climber_2,climber_3
ID,,,,,,,
1,"Eric Wright and Frank Robertson, 2008",2008,Eric Wright and Frank Robertson,Eric Wright;Frank Robertson,Eric Wright,Frank Robertson,None
2,"John Long, Lynn Hill & Joanne Urioste '83",83,"John Long, Lynn Hill & Joanne Urioste",John Long;Lynn Hill;Joanne Urioste,John Long,Lynn Hill,Joanne Urioste
3,"L. Hill, 1993",1993,L. Hill,Lynn Hill,Lynn Hill,None,None


# Normalize year

In [23]:
def normalize_year(y):
    y = int(y)
    if y < 22:
        # it is only 2 digits and passed 2000s
        y += 2000
    elif y < 100:
        # it is only 2 digits and before 2000s
        y += 1900
    return y

In [24]:
df.insert(
    df.columns.tolist().index('year')+1,
    'year_normalized',
    df['year'].map(normalize_year)
)

In [25]:
df

,fa,year,year_normalized,fa_text,climbers,climber_1,climber_2,climber_3
ID,,,,,,,,
1,"Eric Wright and Frank Robertson, 2008",2008,2008,Eric Wright and Frank Robertson,Eric Wright;Frank Robertson,Eric Wright,Frank Robertson,None
2,"John Long, Lynn Hill & Joanne Urioste '83",83,1983,"John Long, Lynn Hill & Joanne Urioste",John Long;Lynn Hill;Joanne Urioste,John Long,Lynn Hill,Joanne Urioste
3,"L. Hill, 1993",1993,1993,L. Hill,Lynn Hill,Lynn Hill,None,None


# Save output

In [29]:
!mkdir -p data

In [30]:
df.to_csv('data/features-extraction-fa.csv', 
          index=True)